In [1]:
import torch
from torch import nn
import numpy as np
from lgg_model import *
from gensim.models import Word2Vec

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
lgg_model_path = input("请输入想使用的语言模型(无需添加后缀)：")
lgg_model_path = 'lgg_model_paths/' + lgg_model_path
lgg_model = torch.load(lgg_model_path)
lgg_model.eval()

LSTM_enhanced(
  (Embedding): Embedding(942, 100)
  (LSTM): LSTM(100, 100, num_layers=2, batch_first=True, dropout=0.5)
  (Linear): Linear(in_features=100, out_features=942, bias=True)
)

In [3]:
word_model_path = input("请使用想使用的词汇库(无需添加后缀)：")
word_model_path = 'word_model_paths/' + word_model_path
word_model = Word2Vec.load(word_model_path)
wv = word_model.wv

In [4]:
words = input("请输入初始文本：")
del_lst = []
lst = list(words)

for i in lst:
    if i not in wv.key_to_index:
        del_lst.append(i)
for i in del_lst:
    lst.remove(i)

data = np.array([])
for i in lst:
    data = np.append(data, wv.key_to_index[i])

count = int(input("请输入想要生成的字(词)数："))

for i in lst:
    print(i, end='')

for i in range(count):
    data = np.stack((data,))
    x = torch.Tensor(data)
    x = x.to(torch.long)
    y = lgg_model(x)[0][-1]
    p = y.detach().numpy()
    p = softmax(p)

    idx = np.random.choice(np.arange(len(wv)), p=p)
    new_word = wv.index_to_key[idx]
    print(new_word, end='')

    lst.append(new_word)
    data = np.append(data, idx)

print('\nGeneration finished.')



同志们，朋友们：
　　中国共产党根基在人民、血脉在人民、力量中国人民对美好生活的向往不挠。要旗帜鲜明反对历史虚无主义，并在新时代新的文艺评论，进更具有深厚的天下情怀和增成文明建设、国防和军队建设、维护国家安全、坚持创新、坚持新民主主义革命伟大胜利”。阐明这一时期党面临的主要任务是，对全面深化改革开放和社会主义现代化建设新时期党面临的形势、提炼主题、取题材，展现中华民族人的血利脉，实现从中国共产党根基在人民的怒辛气、变化、科技、生机勃勃的历史性跨越，为实现中华民族伟大复兴奠定根本政治前提和制度基础。中国共产党员年来，围绕中心、服务大局，真情、正义、创新生命的观点来看待这是党的十八大以来党中央号召你们，牢记初心使命，永远在路上，解决法创新、维护党的团结、维护党中央权威和集中统一领导，继续发展当代中华文艺立场，响时代的号角。古人说：
　　同志们、朋友们！
　　新的征程上，我们必须坚持马克思列宁主义同实践远大历史决议基本都解决了，其基本论述和结论相衔接，体现了党和国家事业发展的光彩，为了解对对人类进步事业、对马克思主义基本原理同中国具有几化建设和人民一道感受时代的脉搏、生命的工作。练就高超艺术水平非朝夕之功，需要专心致志、朝夕、久久为功。如果只想走径、求速成、逐虚名，想、干部分“夺取中国共产党矢志不渝是社会主义文明强军指寻同中国历史，这次全会决议要把着力点放在总结党的百年奋斗的历史意义，总结党和国家事业取得的历史性成就、发生前途和命运，把力量在文艺创新是文艺工作者要讲品位、讲格调的。科技自政策，军力和平、自信的态度，择取最能代表中国变革和大大优秀作品滋养人民的富多向人类进步作出了不可灭的贡献。1840年片战争以后，党的宗旨，永远保持同人民群众的血肉联系，战胜一系列严挑战、巩固新生政权，成功完成社会生产力，使命担当，充分体现党在前两个历史决议基本都解决的决议》，对建党以后特别是党和国家事业发展历史，前两个历史决议基本框架和主要内容
　　今年3月已经，是党拥有百年之！党时代中，中国共产党人作出新征程的恢宏气象。实现中华民族伟大复兴提供了充满新的活力的体制保证和快速发展、关系，完整、准确的贯通、道德规范，把艺术创造力和中华文化价值融合起来、健们在总结。
　　——以史为鉴、开创未来，深切怀念为中国革命、建设、改革事业，贯通中华民族伟大复兴的目标，同时也必须准备付出更为艰巨、更为艰苦